In [1]:
import pandas as pd
import numpy as np
import re as re
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as mno
from numpy import load

from sklearn.ensemble import *
from sklearn.feature_selection import SelectKBest , f_classif 
from sklearn.linear_model import LinearRegression , LogisticRegression, SGDClassifier, Ridge
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
import lightgbm as lgb
from mlxtend.classifier import StackingCVClassifier

from sklearn.model_selection import RepeatedStratifiedKFold ,cross_val_score
from sklearn.model_selection import  train_test_split , RandomizedSearchCV
from sklearn.pipeline import Pipeline

from sklearn.metrics import f1_score , make_scorer
from scipy.stats import loguniform
from imblearn.under_sampling import RandomUnderSampler

import gc

import warnings
warnings.filterwarnings('ignore')

# Back Order Prediction
    A common challenge in Supply Chain Management , where a company wants to optimize the product storage such that minimum space and maximum profit is gained.
    If a company can predict shortage of a product prior then it can store it and make profits.

<html>
    <h1>Features</h1>
    <body>
        <br>
        <div><b>sku -</b> sku code A stock-keeping unit (SKU) is a scannable bar code.</div><br>
        <div><b>national_inv -</b> Current inventory level of component</div><br>
        <div><b>lead_time -</b>Transit time</div><br>
        <div><b>in_transit_qtry -</b> Quantity in transit</div><br>
        <div><b>forecast_x_month -</b> Forecast sales for the net 3, 6, 9 months</div><br>
        <div><b>sales_x_month -</b> Sales quantity for the prior 1, 3, 6, 9 months</div><br>
        <div><b>min_bank -</b> Minimum recommended amount in stock</div><br>
        <div><b>potential_issue -</b> Indictor variable noting potential issue with item</div><br>
        <div><b>pieces_past_due -</b> Parts overdue from source</div><br>
        <div><b>perf_x_months_avg -</b> Source performance in the last 6 and 12 months</div><br>
        <div><b>local_bo_qty -</b> Amount of stock orders overdue</div><br>
        <div><b>X17-X21 -</b> General Risk Flags</div><br>
        <div><b>went_on_back_order -</b> Product went on backorder (Target Variable)</div><br>
    </body>
</html>

<html>
<body>
<div>

It is a Binary Classification problem , aim of this Case study is to predict whether there is gonna be shortage or not.
</div>
<br>
Target Variable is <b>went_on_backorder</b>
<ul>    
   <li>0 -> no shortage expected</li> 
   <li>1-> expected shortage</li>
</ul>
<body>
</html>

##  Modelling

<html>
    <body>
    <br>
    <b>Metric</b>
    <div>F1-score because  False Positive and False Negative are both useful for making profit.</div><br>
	<b>Modelling</b>
     <div>
        1.Dummy Model<br>
        2.Linear Models(Log Reg , SGDClassifier)<br>
        3.Non Linear Models(Decision Tree)<br>
        4.Ensembles(RF , XGBoost)<br>
    </div>
    </body>
</html>

###  Dummy model

In [1]:
from numpy import load

In [2]:
#These are SMOTE only sampled datasets
X_1 = load('X_1.npy')
y_1 = load('y_1.npy')

In [3]:
#These are SMOTE with undersampling datasets
X_test = load('X_2.npy')
y_test = load('y_2.npy')

In [5]:
print(type(X_test))

<class 'numpy.ndarray'>


In [16]:
from sklearn.dummy import DummyClassifier
model_d = DummyClassifier(strategy="uniform")
model_d.fit(X_1,y_1)
y_pred = model_d.predict(X_test_fs)
f1_score_dummy = f1_score(test_y,y_pred)
print("F1_score using SMOTE for oversampling: {}".format(f1_score_dummy))

F1_score using SMOTE for oversampling: 0.013526145562639048


In [17]:
#SMOTE with undersampling
model_d2 = DummyClassifier(strategy="uniform")
model_d2.fit(X_2,y_2)
y_pred2 = model_d2.predict(X_test_fs)
f1_score_dummy1 = f1_score(test_y,y_pred2)
print("F1_score using SMOTE for oversampling: {}".format(f1_score_dummy1))

F1_score using SMOTE for oversampling: 0.013230884557721138


### Linear Models

#### Logistic Regression

In [18]:
f1_scorer = make_scorer(f1_score)
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(model,grid, n_jobs=-1, cv=cv, scoring= f1_scorer,error_score=0)
grid_result = r_cv.fit(X_1, y_1)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.643162 using {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.01}
0.643012 (0.000723) with: {'solver': 'newton-cg', 'penalty': 'l2', 'C': 10}
0.643079 (0.000751) with: {'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.01}
0.643154 (0.000799) with: {'solver': 'liblinear', 'penalty': 'l2', 'C': 10}
0.643092 (0.000852) with: {'solver': 'liblinear', 'penalty': 'l2', 'C': 100}
0.643162 (0.000856) with: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.01}
0.643141 (0.000783) with: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.1}
0.566548 (0.061833) with: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 1.0}
0.643014 (0.000723) with: {'solver': 'newton-cg', 'penalty': 'l2', 'C': 1.0}
0.643023 (0.000698) with: {'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.1}
0.643090 (0.000808) with: {'solver': 'liblinear', 'penalty': 'l2', 'C': 1.0}


In [10]:
log_reg = LogisticRegression(solver = 'liblinear', penalty = 'l2', C = 0.01)
log_reg.fit(X_1,y_1)
y_pred = log_reg.predict(X_test)
log_reg_f1 = f1_score(y_test,y_pred,average='macro')
print(log_reg_f1)

0.5979903222563487


In [2]:
log_reg_f1 = 0.5979903222563487

In [37]:
f1_scorer = make_scorer(f1_score)
model1_lr = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(model1_lr,grid, n_jobs=-1, cv=cv, scoring= f1_scorer,error_score=0)
grid_result = r_cv.fit(X_2, y_2)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.641720 using {'solver': 'liblinear', 'penalty': 'l2', 'C': 10}
0.641720 (0.001042) with: {'solver': 'liblinear', 'penalty': 'l2', 'C': 10}
0.545748 (0.057331) with: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}
0.641718 (0.000899) with: {'solver': 'liblinear', 'penalty': 'l2', 'C': 100}
0.559142 (0.060236) with: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
0.641651 (0.000866) with: {'solver': 'newton-cg', 'penalty': 'l2', 'C': 1.0}
0.541897 (0.052710) with: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 1.0}
0.641664 (0.000866) with: {'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.1}
0.641654 (0.000812) with: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.01}
0.530849 (0.045216) with: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 100}
0.525901 (0.040471) with: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.1}


In [6]:
#Taking a sample of this data set to work with SVM classifiers
X_1s = X_1[:1000000]
y_1s = y_1[:1000000]
X_2s = X_2[:1000000]
y_2s = y_2[:1000000]

### Non Linear Models

#### Decisiton Tree

In [5]:
f1_scorer = make_scorer(f1_score)

model_dt1 = DecisionTreeClassifier()

params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(model_dt1, params, n_jobs=-1, cv=cv, scoring=f1_scorer,error_score=0)
grid_result = r_cv.fit(X_1, y_1)
# summarize results`
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.778362 using {'min_samples_leaf': 10, 'max_depth': 20, 'criterion': 'gini'}
0.537912 (0.001260) with: {'min_samples_leaf': 100, 'max_depth': 3, 'criterion': 'gini'}
0.240704 (0.001188) with: {'min_samples_leaf': 20, 'max_depth': 2, 'criterion': 'gini'}
0.773723 (0.001051) with: {'min_samples_leaf': 50, 'max_depth': 20, 'criterion': 'gini'}
0.778362 (0.001010) with: {'min_samples_leaf': 10, 'max_depth': 20, 'criterion': 'gini'}
0.240704 (0.001188) with: {'min_samples_leaf': 20, 'max_depth': 2, 'criterion': 'entropy'}
0.240704 (0.001188) with: {'min_samples_leaf': 50, 'max_depth': 2, 'criterion': 'entropy'}
0.624105 (0.002672) with: {'min_samples_leaf': 50, 'max_depth': 10, 'criterion': 'entropy'}
0.540683 (0.001190) with: {'min_samples_leaf': 5, 'max_depth': 3, 'criterion': 'entropy'}
0.538765 (0.001284) with: {'min_samples_leaf': 10, 'max_depth': 5, 'criterion': 'gini'}
0.541678 (0.001214) with: {'min_samples_leaf': 5, 'max_depth': 5, 'criterion': 'entropy'}


In [17]:
dt = DecisionTreeClassifier(min_samples_leaf = 10, max_depth = 20, criterion = 'gini')
dt.fit(X_1,y_1)
y_pred = dt.predict(X_test)
dt_f1 = f1_score(y_true=y_test,y_pred=y_pred,average="macro")

In [ ]:
print(dt_f1)

In [3]:
dt_f1 = 0.7806553008736472

In [7]:
model_dt2 = DecisionTreeClassifier()

params = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(model_dt2, params, n_jobs=-1, cv=cv, scoring=f1_scorer,error_score=0)
grid_result = r_cv.fit(X_2, y_2)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.774104 using {'min_samples_leaf': 20, 'max_depth': 20, 'criterion': 'gini'}
0.623425 (0.004512) with: {'min_samples_leaf': 20, 'max_depth': 10, 'criterion': 'entropy'}
0.770626 (0.001474) with: {'min_samples_leaf': 50, 'max_depth': 20, 'criterion': 'gini'}
0.538490 (0.001617) with: {'min_samples_leaf': 5, 'max_depth': 3, 'criterion': 'entropy'}
0.547005 (0.003295) with: {'min_samples_leaf': 10, 'max_depth': 5, 'criterion': 'gini'}
0.765163 (0.001341) with: {'min_samples_leaf': 100, 'max_depth': 20, 'criterion': 'gini'}
0.765340 (0.001139) with: {'min_samples_leaf': 100, 'max_depth': 20, 'criterion': 'entropy'}
0.623178 (0.004501) with: {'min_samples_leaf': 50, 'max_depth': 10, 'criterion': 'entropy'}
0.538196 (0.001598) with: {'min_samples_leaf': 100, 'max_depth': 3, 'criterion': 'gini'}
0.772233 (0.001196) with: {'min_samples_leaf': 20, 'max_depth': 20, 'criterion': 'entropy'}
0.774104 (0.001490) with: {'min_samples_leaf': 20, 'max_depth': 20, 'criterion': 'gini'}


### Ensembles

#### Bagging 

In [8]:
# define models and parameters
f1_scorer = make_scorer(f1_score)

bagging1 = BaggingClassifier()
n_estimators = [10, 100, 1000]
# define grid search
grid = dict(n_estimators=n_estimators)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(bagging1, grid, n_jobs=-1, cv=cv, scoring=f1_scorer,error_score=0)
grid_result = r_cv.fit(X_1s, y_1s)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.158604 using {'n_estimators': 100}
0.147559 (0.017059) with: {'n_estimators': 10}
0.158604 (0.016763) with: {'n_estimators': 100}
0.158181 (0.016835) with: {'n_estimators': 1000}


In [9]:
# define models and parameters
bagging2 = BaggingClassifier()
n_estimators = [10, 100, 1000]
# define grid search
grid = dict(n_estimators=n_estimators)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(bagging2, grid, n_jobs=-1, cv=cv, scoring=f1_scorer,error_score=0)
grid_result = r_cv.fit(X_2s, y_2s)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.000000 using {'n_estimators': 10}
0.000000 (0.000000) with: {'n_estimators': 10}
0.000000 (0.000000) with: {'n_estimators': 100}
0.000000 (0.000000) with: {'n_estimators': 1000}


#### Random Forest

In [10]:
rf1 = RandomForestClassifier()
n_estimators = [10, 100, 1000]
max_features = ['sqrt', 'log2']
# define grid search
grid = dict(n_estimators=n_estimators,max_features=max_features)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(rf1, grid, n_jobs=-1, cv=cv, scoring=f1_scorer,error_score=0)
grid_result = r_cv.fit(X_1s, y_1s)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.155176 using {'n_estimators': 100, 'max_features': 'log2'}
0.143770 (0.016685) with: {'n_estimators': 10, 'max_features': 'sqrt'}
0.154178 (0.016223) with: {'n_estimators': 100, 'max_features': 'sqrt'}
0.153327 (0.017025) with: {'n_estimators': 1000, 'max_features': 'sqrt'}
0.146845 (0.018499) with: {'n_estimators': 10, 'max_features': 'log2'}
0.155176 (0.015976) with: {'n_estimators': 100, 'max_features': 'log2'}
0.153523 (0.016242) with: {'n_estimators': 1000, 'max_features': 'log2'}


In [11]:
rf2 = RandomForestClassifier()
n_estimators = [10, 100, 1000]
max_features = ['sqrt', 'log2']
# define grid search
grid = dict(n_estimators=n_estimators,max_features=max_features)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(rf2, grid, n_jobs=-1, cv=cv, scoring=f1_scorer,error_score=0)
grid_result = r_cv.fit(X_2s, y_2s)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.000000 using {'n_estimators': 10, 'max_features': 'sqrt'}
0.000000 (0.000000) with: {'n_estimators': 10, 'max_features': 'sqrt'}
0.000000 (0.000000) with: {'n_estimators': 100, 'max_features': 'sqrt'}
0.000000 (0.000000) with: {'n_estimators': 1000, 'max_features': 'sqrt'}
0.000000 (0.000000) with: {'n_estimators': 10, 'max_features': 'log2'}
0.000000 (0.000000) with: {'n_estimators': 100, 'max_features': 'log2'}
0.000000 (0.000000) with: {'n_estimators': 1000, 'max_features': 'log2'}


#### Boosting

In [6]:
# define models and parameters
f1_scorer = make_scorer(f1_score)

xgboost1 = GradientBoostingClassifier()
n_estimators = [10, 100, 1000]
learning_rate = [0.001, 0.01, 0.1]
subsample = [0.5, 0.7, 1.0]
max_depth = [3, 7, 9]
# define grid search
grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, subsample=subsample, max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(xgboost1, grid, n_jobs=-1, cv=cv, scoring=f1_scorer,error_score=0)
grid_result = r_cv.fit(X_1s, y_1s)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.130724 using {'subsample': 0.7, 'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.1}
0.000000 (0.000000) with: {'subsample': 1.0, 'n_estimators': 10, 'max_depth': 9, 'learning_rate': 0.01}
0.001272 (0.001805) with: {'subsample': 0.5, 'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.001}
0.130724 (0.017961) with: {'subsample': 0.7, 'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.1}
0.014637 (0.008335) with: {'subsample': 0.7, 'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.01}
0.000870 (0.002383) with: {'subsample': 0.5, 'n_estimators': 10, 'max_depth': 3, 'learning_rate': 0.1}
0.000000 (0.000000) with: {'subsample': 1.0, 'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.001}
0.000000 (0.000000) with: {'subsample': 0.5, 'n_estimators': 10, 'max_depth': 7, 'learning_rate': 0.001}
0.025677 (0.010943) with: {'subsample': 1.0, 'n_estimators': 1000, 'max_depth': 9, 'learning_rate': 0.001}
0.064510 (0.016277) with: {'subsample': 0.5, 'n_estimators':

In [ ]:
# define models and parameters
f1_scorer = make_scorer(f1_score)

xgboost1 = GradientBoostingClassifier()
n_estimators = [10, 100, 1000]
learning_rate = [0.001, 0.01, 0.1]
subsample = [0.5, 0.7, 1.0]
max_depth = [3, 7, 9]
# define grid search
grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, subsample=subsample, max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(xgboost1, grid, n_jobs=-1, cv=cv, scoring=f1_scorer,error_score=0)
grid_result = r_cv.fit(X_1, y_1)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [5]:
xgboost1 = GradientBoostingClassifier(subsample= 0.7, n_estimators= 1000, max_depth= 9, learning_rate=0.1)
xgboost1.fit(X_1,y_1)
y_pred = xgboost1.predict(X_test)
xgb_f1_score = f1_score(y_true=y_test,y_pred=y_pred,average="macro")

In [6]:
print(xgb_f1_score)

0.82302628606276


In [7]:
xgb_f1_score =0.8231992657663574

In [8]:
import joblib
filename = 'final_model.sav'
joblib.dump(xgboost1, filename)

['final_model.sav']

####Order of features after feature selection####<br>
0         lead_time     
1    in_transit_qty       
2          min_bank       
3   potential_issue     
4  perf_6_month_avg     
5      local_bo_qty     
6         deck_risk     
7     oe_constraint      
8         ppap_risk      
9     stop_auto_buy       
##################################################

In [24]:
print(X_test[123])

[2.   0.   0.   0.   0.95 0.   1.   0.   0.   1.  ]


In [25]:
print(xgboost1.predict(X_test[123].reshape(1,-1))[0])

1.0


In [ ]:
# define models and parameters
f1_scorer = make_scorer(f1_score,average='macro')

xgboost1 = GradientBoostingClassifier()
n_estimators = [10, 100, 1000]
learning_rate = [0.001, 0.01, 0.1]
subsample = [0.5, 0.7, 1.0]
max_depth = [3, 7, 9]
# define grid search
grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, subsample=subsample, max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(xgboost1, grid, n_jobs=-1, cv=cv, scoring=f1_scorer,error_score=0)
grid_result = r_cv.fit(X_1, y_1)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
xgboost1 = GradientBoostingClassifier()

#### LightGBM

In [24]:
import lightgbm as lgb

In [9]:
f1_scorer = make_scorer(f1_score,average='macro')

grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'num_leaves': [90,200],
    'boosting_type' : ['gbdt'],
    'max_depth' : [5,6,7,8],
    'random_state' : [501], 
    'colsample_bytree' : [0.5,0.7],
    'subsample' : [0.5,0.7]
    }

lgb_1 = lgb.LGBMClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(lgb_1, grid, n_jobs=-1, cv=cv, scoring=f1_scorer,error_score=0)
grid_result = r_cv.fit(X_1, y_1)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.752849 using {'subsample': 0.5, 'random_state': 501, 'num_leaves': 200, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}
0.746519 (0.002141) with: {'subsample': 0.5, 'random_state': 501, 'num_leaves': 200, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 0.7, 'boosting_type': 'gbdt'}
0.713210 (0.000656) with: {'subsample': 0.7, 'random_state': 501, 'num_leaves': 90, 'max_depth': 7, 'learning_rate': 0.01, 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}
0.683763 (0.001092) with: {'subsample': 0.7, 'random_state': 501, 'num_leaves': 200, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 0.7, 'boosting_type': 'gbdt'}
0.752849 (0.002293) with: {'subsample': 0.5, 'random_state': 501, 'num_leaves': 200, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}
0.718409 (0.000977) with: {'subsample': 0.7, 'random_state': 501, 'num_leaves': 90, 'max_depth': 8, 'learning_rate': 0.01, 'colsample_bytre

In [25]:
lgb_clf = lgb.LGBMClassifier(subsample= 0.5, random_state = 501, num_leaves= 200, 
                   max_depth = 7, learning_rate = 0.1, colsample_bytree = 0.5, boosting_type = 'gbdt')
lgb_clf.fit(X_1,y_1)
y_pred = lgb_clf.predict(X_test)
lgb_f1 = f1_score(y_true = y_test,y_pred = y_pred,average='macro')

In [27]:
print(lgb_f1)

0.7539856580054936


In [5]:
lgb_f1 = 0.7539856580054936

In [10]:
f1_scorer = make_scorer(f1_score)

grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'num_leaves': [90,200],
    'boosting_type' : ['gbdt'],
    'max_depth' : [5,6,7,8],
    'random_state' : [501], 
    'colsample_bytree' : [0.5,0.7],
    'subsample' : [0.5,0.7]
    }

lgb_1 = lgb.LGBMClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(lgb_1, grid, n_jobs=-1, cv=cv, scoring=f1_scorer,error_score=0)
grid_result = r_cv.fit(X_1, y_1)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.747126 using {'subsample': 0.7, 'random_state': 501, 'num_leaves': 90, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 0.7, 'boosting_type': 'gbdt'}
0.747126 (0.002102) with: {'subsample': 0.7, 'random_state': 501, 'num_leaves': 90, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 0.7, 'boosting_type': 'gbdt'}
0.668578 (0.001246) with: {'subsample': 0.7, 'random_state': 501, 'num_leaves': 90, 'max_depth': 5, 'learning_rate': 0.001, 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}
0.700692 (0.001301) with: {'subsample': 0.5, 'random_state': 501, 'num_leaves': 90, 'max_depth': 7, 'learning_rate': 0.01, 'colsample_bytree': 0.7, 'boosting_type': 'gbdt'}
0.676374 (0.003244) with: {'subsample': 0.5, 'random_state': 501, 'num_leaves': 90, 'max_depth': 7, 'learning_rate': 0.001, 'colsample_bytree': 0.7, 'boosting_type': 'gbdt'}
0.667538 (0.001621) with: {'subsample': 0.7, 'random_state': 501, 'num_leaves': 90, 'max_depth': 6, 'learning_rate': 0.001, 'colsample_bytre

In [13]:
f1_scorer = make_scorer(f1_score)

grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'num_leaves': [90,200],
    'boosting_type' : ['gbdt'],
    'max_depth' : [5,6,7,8],
    'random_state' : [501], 
    'colsample_bytree' : [0.5,0.7],
    'subsample' : [0.5,0.7]
    }

lgb_2 = lgb.LGBMClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(lgb_2, grid, n_jobs=-1, cv=cv, scoring=f1_scorer,error_score=0)
grid_result = r_cv.fit(X_2, y_2)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.763942 using {'subsample': 0.7, 'random_state': 501, 'num_leaves': 90, 'max_depth': 8, 'learning_rate': 0.1, 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}
0.649445 (0.001883) with: {'subsample': 0.5, 'random_state': 501, 'num_leaves': 200, 'max_depth': 5, 'learning_rate': 0.001, 'colsample_bytree': 0.7, 'boosting_type': 'gbdt'}
0.675832 (0.002196) with: {'subsample': 0.7, 'random_state': 501, 'num_leaves': 200, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}
0.720564 (0.001046) with: {'subsample': 0.5, 'random_state': 501, 'num_leaves': 90, 'max_depth': 8, 'learning_rate': 0.01, 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}
0.713509 (0.001623) with: {'subsample': 0.7, 'random_state': 501, 'num_leaves': 200, 'max_depth': 7, 'learning_rate': 0.01, 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}
0.700652 (0.001491) with: {'subsample': 0.5, 'random_state': 501, 'num_leaves': 90, 'max_depth': 6, 'learning_rate': 0.01, 'colsample_byt

In [11]:
f1_scorer = make_scorer(f1_score,average='macro')

grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'num_leaves': [90,200],
    'boosting_type' : ['gbdt'],
    'max_depth' : [5,6,7,8],
    'random_state' : [501], 
    'colsample_bytree' : [0.5,0.7],
    'subsample' : [0.5,0.7]
    }

lgb_2 = lgb.LGBMClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
r_cv = RandomizedSearchCV(lgb_2, grid, n_jobs=-1, cv=cv, scoring=f1_scorer,error_score=0)
grid_result = r_cv.fit(X_2, y_2)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.771447 using {'subsample': 0.7, 'random_state': 501, 'num_leaves': 200, 'max_depth': 8, 'learning_rate': 0.1, 'colsample_bytree': 0.7, 'boosting_type': 'gbdt'}
0.704746 (0.000969) with: {'subsample': 0.7, 'random_state': 501, 'num_leaves': 200, 'max_depth': 7, 'learning_rate': 0.001, 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}
0.745352 (0.001950) with: {'subsample': 0.5, 'random_state': 501, 'num_leaves': 200, 'max_depth': 6, 'learning_rate': 0.1, 'colsample_bytree': 0.7, 'boosting_type': 'gbdt'}
0.753403 (0.001615) with: {'subsample': 0.5, 'random_state': 501, 'num_leaves': 90, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 0.5, 'boosting_type': 'gbdt'}
0.771447 (0.001831) with: {'subsample': 0.7, 'random_state': 501, 'num_leaves': 200, 'max_depth': 8, 'learning_rate': 0.1, 'colsample_bytree': 0.7, 'boosting_type': 'gbdt'}
0.759066 (0.001580) with: {'subsample': 0.5, 'random_state': 501, 'num_leaves': 200, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytre

#### Making a stacking classifier


<ol>
<li>Using dataset A i.e SMOTE without under sampling.</li>
<li><ul>Base Learners used are following with pre trained hyperparameters:
                <li>Logistic Regression</li>
                <li>GradientBoostingClassifier</li>
                <li>DecisionTreeClassifier</li>
                <li>LGBM</li></ul></li>
 <li>Using Logistic Regression as meta learner.</li></ol></html>

In [9]:
#These are SMOTE only sampled datasets
X_train = load('X_1.npy')
y_train = load('y_1.npy')
X_test  = load('X_1_test.npy')
y_test = load('y_1_test.npy')

In [11]:
print(X_train.shape)
print(X_test.shape)

(2514850, 10)
(421965, 10)


In [12]:
clf1 = LogisticRegression(solver='liblinear', penalty='l2', C = 0.01)
clf2 = DecisionTreeClassifier(min_samples_leaf= 10, max_depth= 20, criterion ='gini')
clf3 = GradientBoostingClassifier(subsample = 0.7, n_estimators = 1000, max_depth = 9, learning_rate = 0.1)
level_1 = LogisticRegression()

####  Stacking_Model 1

In [20]:
base_learners = [
                 ('log_reg', LogisticRegression(solver='liblinear', penalty='l2', C = 0.01)),
                 ('dt', DecisionTreeClassifier(min_samples_leaf= 10, max_depth= 20, criterion ='gini')) ,
                 ('gbdt',GradientBoostingClassifier(subsample = 0.7, n_estimators = 1000, max_depth = 9, learning_rate = 0.1))
                ]

# Initialize Stacking Classifier with the Meta Learner
clf = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())

# Extract score
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
f1_score(y_true=y_test,y_pred=y_pred,average="macro")

0.4733751633061474

#### Stacking_Model 2

In [21]:
base_learners = [
                 ('log_reg', LogisticRegression(solver='liblinear', penalty='l2', C = 0.01)),
                 ('dt', DecisionTreeClassifier(min_samples_leaf= 10, max_depth= 20, criterion ='gini')) ,
                 ('gbdt',GradientBoostingClassifier(subsample = 0.7, n_estimators = 1000, max_depth = 9, learning_rate = 0.1))
                ]

# Initialize Stacking Classifier with the Meta Learner
clf = StackingClassifier(estimators=base_learners,
                         final_estimator=LogisticRegression(solver='liblinear', penalty='l2', C = 0.01))

# Extract score
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
f1_score(y_true=y_test,y_pred=y_pred,average="macro")

0.4736268603406309

#### Stacking_Model 3

In [25]:
base_learners = [
                 ('log_reg', LogisticRegression(solver='liblinear', penalty='l2', C = 0.01)),
                 ('dt', DecisionTreeClassifier(min_samples_leaf= 10, max_depth= 20, criterion ='gini')) ,
                 ('gbdt',GradientBoostingClassifier(subsample = 0.7, n_estimators = 1000, max_depth = 9, learning_rate = 0.1)),
                 ('lgb',lgb.LGBMClassifier(subsample= 0.5, random_state = 501, num_leaves= 200, 
                   max_depth = 7, learning_rate = 0.1, colsample_bytree = 0.5, boosting_type = 'gbdt'))
                ]

# Initialize Stacking Classifier with the Meta Learner
clf = StackingClassifier(estimators=base_learners,
                         final_estimator=GradientBoostingClassifier(subsample = 0.7, 
                                                                    n_estimators = 1000,
                                                                    max_depth = 9, 
                                                                    learning_rate = 0.1))

# Extract score
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
f1_score(y_true=y_test,y_pred=y_pred,average="macro")

0.4693022759166582

#### Stacking_Model 4

##### Define the Level 0 classifiers

In [27]:
clf1 = LogisticRegression(solver='liblinear', penalty='l2', C = 0.01)
clf2 = DecisionTreeClassifier(min_samples_leaf= 10, max_depth= 20, criterion ='gini')
clf3 = GradientBoostingClassifier(subsample = 0.7, n_estimators = 1000, max_depth = 9, learning_rate = 0.1)
clf4 = lgb.LGBMClassifier(subsample= 0.5, random_state = 501, num_leaves= 200, 
                   max_depth = 7, learning_rate = 0.1, colsample_bytree = 0.5, boosting_type = 'gbdt')

#### Define the Level 1 classifiers or Meta learner

In [31]:
sclf = StackingCVClassifier(classifiers = [clf1, clf2, clf3, clf4],
                            cv = 5,
                            meta_classifier = LogisticRegression(solver='liblinear', penalty='l2', C = 0.01))

In [32]:
classifiers = {"Log_Reg": clf1,
               "CART": clf2,
               "GBDT": clf3,
               "LGBM": clf4,
               "Stacking_Classifier": sclf}

#### Train the classifiers

In [33]:
for key in classifiers:
    # Get classifier
    classifier = classifiers[key]
    
    # Fit classifier
    classifier.fit(X_train, y_train)
        
    # Save fitted classifier
    classifiers[key] = classifier

#### Make a dataframe for storing predicted class and f1_score

In [42]:
results = pd.DataFrame(columns=['Classifier','F1_score'])
for key in classifiers:
    # Make prediction on test set
    y_pred = classifiers[key].predict(X_test)
    # Save results in pandas dataframe object
    results.loc[len(results.index)] = [key,f1_score(y_test,y_pred,average='macro')]    

Log_Reg->0.337264996214767
CART->0.4503255702360726
GBDT->0.467952782655212
LGBM->0.4432023048619828
Stacking_Classifier->0.467923417937662


In [43]:
results.head()

,Classifier,F1_score
0,Log_Reg,0.337265
1,CART,0.450326
2,GBDT,0.467953
3,LGBM,0.443202
4,Stacking_Classifier,0.467923


### Conclusion

In [6]:
from prettytable import PrettyTable
  
# Specify the Column Names while initializing the Table
myTable = PrettyTable(["Classifier Name", "F1_Score-macro","Sampling Used"])
  
# Add rows
myTable.add_row(["Logistic Reression",log_reg_f1 ,"SMOTE"])
myTable.add_row(["Decision Tree", dt_f1,"SMOTE"])
myTable.add_row(["XGBoost", xgb_f1_score,"SMOTE"])
myTable.add_row(["LGBM", lgb_f1,"SMOTE"])
myTable.add_row(["Stacking Model 1", 0.4733751633061474,"SMOTE"])
myTable.add_row(["Stacking Model 2", 0.4736268603406309,"SMOTE"])
myTable.add_row(["Stacking Model 3", 0.4693022759166582,"SMOTE"])
myTable.add_row(["Stacking Model 4", 0.467923417937662,"SMOTE"])
myTable.add_row(["Kaggle Public Score(Top 20%)", 0.93301,"XGBoost with all features and sku as index"])
myTable.add_row(["Kaggle Private Score(Top 7%)",0.93901,"XGBoost with all features and sku as index"])


print(myTable)

+------------------------------+--------------------+--------------------------------------------+
|       Classifier Name        |   F1_Score-macro   |               Sampling Used                |
+------------------------------+--------------------+--------------------------------------------+
|      Logistic Reression      | 0.5979903222563487 |                   SMOTE                    |
|        Decision Tree         | 0.7806553008736472 |                   SMOTE                    |
|           XGBoost            | 0.8231992657663574 |                   SMOTE                    |
|             LGBM             | 0.7539856580054936 |                   SMOTE                    |
|       Stacking Model 1       | 0.4733751633061474 |                   SMOTE                    |
|       Stacking Model 2       | 0.4736268603406309 |                   SMOTE                    |
|       Stacking Model 3       | 0.4693022759166582 |                   SMOTE                    |
|       St